In [1]:
import numpy as np
import torch
import copy
from metabci.brainda.algorithms.deep_learning import EEGNet_Sparse
from metabci.brainda.algorithms.utils.model_selection import (
    set_random_seeds,
    generate_kfold_indices, match_kfold_indices)
from metabci.brainda.datasets import Wang2016, BETA
# 三个SSVEP Datasets：Nakanishi2015、Wang2016、BETA
from metabci.brainda.paradigms import SSVEP
from skorch.helper import predefined_split

In [2]:
#**************************************************
# Benchmark数据集读取处理
#**************************************************
Bench_dataset = Wang2016()
subject_list = list(range(1, 21))  # 被试编号从1到20
for s in subject_list:
    Bench_dataset.data_path(subject=s, path="E:\\MetaBCI-master\\mne_data")  # 依次为每个被试设置路径
events = Bench_dataset.events.keys()
freq_list = [str(Bench_dataset.get_freq(event)) for event in events]  # 获得所有刺激的频率
freq_map = {i: freq for i, freq in enumerate(freq_list)}  # 标签到频率的映射

Bench_subjects = list(range(1, 21))  # 使用S1–S20被试进行训练
Bench_paradigm = SSVEP(
    channels=['POZ', 'PZ', 'PO3', 'PO5', 'PO4', 'PO6', 'O1', 'OZ', 'O2'],  # 选择电极通道
    intervals=[(0.14, 1.14)],  # 分析时间窗最长-0.5-5.5s ,0.0是刺激开始
    events=freq_list,  # 选择所有刺激频率
    srate=250  # 采样率
)

# add 5-90Hz bandpass filter in raw hook
# 对原始EEG信号做5-90Hz带通滤波。
def raw_hook(raw, caches):
    raw.filter(5, 90, l_trans_bandwidth=2, h_trans_bandwidth=5, phase='zero-double')
    caches['raw_stage'] = caches.get('raw_stage', -1) + 1
    return raw, caches

Bench_paradigm.register_raw_hook(raw_hook)



# 获取微调数据
X_Bench, y_Bench, meta_Bench = Bench_paradigm.get_data(
    Bench_dataset,
    subjects=Bench_subjects,
    return_concat=True,
    n_jobs=None,
    verbose=False
)

--------ssssss, /upload/yijun/S1.mat.7z
--------ssssss, /upload/yijun/S2.mat.7z
--------ssssss, /upload/yijun/S3.mat.7z
--------ssssss, /upload/yijun/S4.mat.7z
--------ssssss, /upload/yijun/S5.mat.7z
--------ssssss, /upload/yijun/S6.mat.7z
--------ssssss, /upload/yijun/S7.mat.7z
--------ssssss, /upload/yijun/S8.mat.7z
--------ssssss, /upload/yijun/S9.mat.7z
--------ssssss, /upload/yijun/S10.mat.7z
--------ssssss, /upload/yijun/S11.mat.7z
--------ssssss, /upload/yijun/S12.mat.7z
--------ssssss, /upload/yijun/S13.mat.7z
--------ssssss, /upload/yijun/S14.mat.7z
--------ssssss, /upload/yijun/S15.mat.7z
--------ssssss, /upload/yijun/S16.mat.7z
--------ssssss, /upload/yijun/S17.mat.7z
--------ssssss, /upload/yijun/S18.mat.7z
--------ssssss, /upload/yijun/S19.mat.7z
--------ssssss, /upload/yijun/S20.mat.7z
--------ssssss, /upload/yijun/S1.mat.7z
--------ssssss, /upload/yijun/S2.mat.7z
--------ssssss, /upload/yijun/S3.mat.7z
--------ssssss, /upload/yijun/S4.mat.7z
--------ssssss, /upload/yijun

In [11]:
#**************************************************
# Benchmark数据集  EEGNET网络
# !python3.8 Bench_freezing.py # Bench 1e-4学习率，S1~S20  F1=40,D=2,F2=80,Epsilon=20,shared_ratio=0.3,alpha=1.0
# 所有折准确率: [0.68125, 0.6325, 0.6075, 0.6575, 0.655, 0.64375]
# 平均准确率： 0.6462500000000001
#**************************************************
# 设置device（如果GPU可用则使用GPU，否则使用CPU）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
# 设置随机种子
set_random_seeds(42)
kfold = 6
print(X_Bench.shape)  # (2160, 9, 1250)
indices = generate_kfold_indices(meta_Bench, kfold=kfold)

# 预训练阶段
F1 = 20
D = 2
F2 = 40
n_channels = X_Bench.shape[1] # 输入信号的通道数
n_samples = X_Bench.shape[2] # 输入信号的采样点数，时长*采样率
n_classes = 40
time_kernel_length = 50
shared_ratio = 0
alpha = 0
epsilon = 30

Bench_save_path = "Bench_eegnet_Sparse_dropout_1s_0_0.0.pth"

pre_accs = []
best_test_acc = -1
best_state_dict = None

for k in range(kfold):
    # 初始化模型
    estimator = EEGNet_Sparse(F1,D,F2,n_channels,n_samples,n_classes,time_kernel_length,shared_ratio,alpha,epsilon)
    estimator.set_params(optimizer__lr=1e-3,device=device) # optimizer__lr=3e-3,
    train_ind, validate_ind, test_ind = match_kfold_indices(k, meta_Bench, indices)
    # 合并train_ind和validate_ind作为新的训练集索引
    new_train_ind = np.concatenate([train_ind, validate_ind])
    new_valid_ind = test_ind  # 用test_ind作为验证集
    new_test_ind = test_ind   # 用test_ind作为测试集

    print(f"train_ind数量: {len(new_train_ind)}, validate_ind数量: {len(new_valid_ind)}, test_ind数量: {len(new_test_ind)}")
    valid_ds = torch.utils.data.TensorDataset(
        torch.tensor(X_Bench[new_valid_ind], dtype=torch.float64),
        torch.tensor(y_Bench[new_valid_ind], dtype=torch.long)
    )
    estimator.set_params(train_split=predefined_split(valid_ds))
    estimator.fit(X_Bench[new_train_ind], y_Bench[new_train_ind])
    # 测试集评估
    p_labels = estimator.predict(X_Bench[new_test_ind])
    p_freqs = [freq_map[label] for label in p_labels]
    y_freqs = [freq_map[label] for label in y_Bench[new_test_ind]]
    print("预测频率：", p_freqs)
    print("真实频率：", y_freqs)
    pre_accs.append(np.mean(p_labels == y_Bench[new_test_ind]))
    print(f"第{k+1}折准确率: {pre_accs[-1]}")
    # 保存最优模型
    if pre_accs[-1] > best_test_acc:
        best_test_acc = pre_accs[-1]
        best_state_dict = copy.deepcopy(estimator.module.state_dict())
print(f"所有折准确率: {pre_accs}")
print("平均准确率：", np.mean(pre_accs))
# 保存测试集上最优的模型
if best_state_dict is not None:
    torch.save(best_state_dict, Bench_save_path)

cuda
(4800, 9, 250)
train_ind数量: 4000, validate_ind数量: 800, test_ind数量: 800
  epoch    train_acc    train_loss    valid_acc    valid_loss    cp      lr     dur
-------  -----------  ------------  -----------  ------------  ----  ------  ------
      1       0.0267        3.8666       0.0338        3.6862     +  0.0010  1.0393
      2       0.0325        3.7452       0.0462        3.6678     +  0.0010  0.7442
      3       0.0355        3.7027       0.0612        3.6474     +  0.0010  0.7778
      4       0.0442        3.6699       0.0963        3.6202     +  0.0010  0.7420
      5       0.0563        3.6322       0.1138        3.5884     +  0.0010  0.7734
      6       0.0683        3.6005       0.1363        3.5438     +  0.0010  0.7330
      7       0.0862        3.5454       0.1713        3.4868     +  0.0010  0.7710
      8       0.1172        3.4741       0.2037        3.4059     +  0.0010  0.7608
      9       0.1460        3.3982       0.2275        3.2965     +  0.0010  0.7663


In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
test_subjects = [22,24,27,28,31,34,35] # 测试使用 
pre_save_path = "Bench_eegnet_Sparse_dropout_1s_0_0.0.pth"
# 加载预训练模型
F1 = 20
D = 2
F2 = 40
n_channels = X_Bench.shape[1] # 输入信号的通道数
n_samples = X_Bench.shape[2] # 输入信号的采样点数，时长*采样率
n_classes = 40
time_kernel_length = 50
shared_ratio = 0
alpha = 0
epsilon = 30
estimator = EEGNet_Sparse(F1,D,F2,n_channels,n_samples,n_classes,time_kernel_length,shared_ratio,alpha,epsilon)
estimator.set_params(device=device)
estimator.initialize()  
estimator.module.load_state_dict(torch.load(pre_save_path))
subject_accs = {}

# 逐一验证每个被试
for test_subj in test_subjects:
    # 获取该被试的数据
    X, y, meta = Bench_paradigm.get_data(
        Bench_dataset,
        subjects=[test_subj],
        return_concat=True,
        n_jobs=None,
        verbose=False
    )
    
    # 直接用预训练模型预测
    preds = estimator.predict(X)
    acc = np.mean(preds == y)
    subject_accs[str(test_subj)] = acc

# 先把每个被试的准确率转成百分比并保留一位小数
acc_list = [round(acc * 100, 1) for acc in subject_accs.values()]
# 逐一打印单个被试
for subj, acc in zip(test_subjects, acc_list):
    print(f"subject {subj} 的准确率: {acc:.1f}%")

# 打印所有被试准确率（字典仍是原始小数）
print("所有被试的准确率：", subject_accs)

# 基于百分比后的数值再算平均
mean_acc = np.mean(acc_list)
print(f"所有被试的平均准确率: {mean_acc:.1f}%")

# 额外一行：只显示所有被试准确率（百分数），逗号隔开
print("所有被试准确率（无名称）:", ",".join([f"{acc:.1f}" for acc in acc_list]))

cuda
--------ssssss, /upload/yijun/S22.mat.7z
--------ssssss, /upload/yijun/S24.mat.7z
--------ssssss, /upload/yijun/S27.mat.7z
--------ssssss, /upload/yijun/S28.mat.7z
--------ssssss, /upload/yijun/S31.mat.7z
--------ssssss, /upload/yijun/S34.mat.7z
--------ssssss, /upload/yijun/S35.mat.7z
subject 22 的准确率: 77.1%
subject 24 的准确率: 77.9%
subject 27 的准确率: 83.8%
subject 28 的准确率: 69.6%
subject 31 的准确率: 77.5%
subject 34 的准确率: 67.1%
subject 35 的准确率: 80.4%
所有被试的准确率： {'22': 0.7708333333333334, '24': 0.7791666666666667, '27': 0.8375, '28': 0.6958333333333333, '31': 0.775, '34': 0.6708333333333333, '35': 0.8041666666666667}
所有被试的平均准确率: 76.2%
所有被试准确率（无名称）: 77.1,77.9,83.8,69.6,77.5,67.1,80.4


In [13]:
#**************************************************
# Benchmark数据集  EEGNET网络
# !python3.8 Bench_freezing.py # Bench 1e-4学习率，S1~S20  F1=40,D=2,F2=80,Epsilon=20,shared_ratio=0.3,alpha=1.0
# 所有折准确率: [0.68125, 0.6325, 0.6075, 0.6575, 0.655, 0.64375]
# 平均准确率： 0.6462500000000001
#**************************************************
# 设置device（如果GPU可用则使用GPU，否则使用CPU）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
# 设置随机种子
set_random_seeds(42)
kfold = 6
print(X_Bench.shape)  # (2160, 9, 1250)
indices = generate_kfold_indices(meta_Bench, kfold=kfold)

# 预训练阶段
F1 = 20
D = 2
F2 = 40
n_channels = X_Bench.shape[1] # 输入信号的通道数
n_samples = X_Bench.shape[2] # 输入信号的采样点数，时长*采样率
n_classes = 40
time_kernel_length = 50
shared_ratio = 0.1
alpha = 0
epsilon = 30

Bench_save_path = "Bench_eegnet_Sparse_dropout_1s_0_0.1.pth"

pre_accs = []
best_test_acc = -1
best_state_dict = None

for k in range(kfold):
    # 初始化模型
    estimator = EEGNet_Sparse(F1,D,F2,n_channels,n_samples,n_classes,time_kernel_length,shared_ratio,alpha,epsilon)
    estimator.set_params(optimizer__lr=1e-3,device=device) # optimizer__lr=3e-3,
    train_ind, validate_ind, test_ind = match_kfold_indices(k, meta_Bench, indices)
    # 合并train_ind和validate_ind作为新的训练集索引
    new_train_ind = np.concatenate([train_ind, validate_ind])
    new_valid_ind = test_ind  # 用test_ind作为验证集
    new_test_ind = test_ind   # 用test_ind作为测试集

    print(f"train_ind数量: {len(new_train_ind)}, validate_ind数量: {len(new_valid_ind)}, test_ind数量: {len(new_test_ind)}")
    valid_ds = torch.utils.data.TensorDataset(
        torch.tensor(X_Bench[new_valid_ind], dtype=torch.float64),
        torch.tensor(y_Bench[new_valid_ind], dtype=torch.long)
    )
    estimator.set_params(train_split=predefined_split(valid_ds))
    estimator.fit(X_Bench[new_train_ind], y_Bench[new_train_ind])
    # 测试集评估
    p_labels = estimator.predict(X_Bench[new_test_ind])
    p_freqs = [freq_map[label] for label in p_labels]
    y_freqs = [freq_map[label] for label in y_Bench[new_test_ind]]
    print("预测频率：", p_freqs)
    print("真实频率：", y_freqs)
    pre_accs.append(np.mean(p_labels == y_Bench[new_test_ind]))
    print(f"第{k+1}折准确率: {pre_accs[-1]}")
    # 保存最优模型
    if pre_accs[-1] > best_test_acc:
        best_test_acc = pre_accs[-1]
        best_state_dict = copy.deepcopy(estimator.module.state_dict())
print(f"所有折准确率: {pre_accs}")
print("平均准确率：", np.mean(pre_accs))
# 保存测试集上最优的模型
if best_state_dict is not None:
    torch.save(best_state_dict, Bench_save_path)

cuda
(4800, 9, 250)
train_ind数量: 4000, validate_ind数量: 800, test_ind数量: 800
  epoch    train_acc    train_loss    valid_acc    valid_loss    cp      lr     dur
-------  -----------  ------------  -----------  ------------  ----  ------  ------
      1       0.0290        3.8474       0.0350        3.6863     +  0.0010  1.0230
      2       0.0307        3.7413       0.0413        3.6688     +  0.0010  0.7834
      3       0.0348        3.7021       0.0663        3.6493     +  0.0010  0.8145
      4       0.0445        3.6704       0.0838        3.6232     +  0.0010  0.7372
      5       0.0558        3.6369       0.1087        3.5927     +  0.0010  0.7464
      6       0.0707        3.6044       0.1300        3.5511     +  0.0010  0.7733
      7       0.0830        3.5543       0.1613        3.4990     +  0.0010  0.7892
      8       0.1145        3.4857       0.1900        3.4224     +  0.0010  0.8128
      9       0.1405        3.4220       0.2100        3.3236     +  0.0010  0.7547


In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
test_subjects = [22,24,27,28,31,34,35] # 测试使用 
pre_save_path = "Bench_eegnet_Sparse_dropout_1s_0_0.1.pth"
# 加载预训练模型
F1 = 20
D = 2
F2 = 40
n_channels = X_Bench.shape[1] # 输入信号的通道数
n_samples = X_Bench.shape[2] # 输入信号的采样点数，时长*采样率
n_classes = 40
time_kernel_length = 50
shared_ratio = 0.1
alpha = 0
epsilon = 30
estimator = EEGNet_Sparse(F1,D,F2,n_channels,n_samples,n_classes,time_kernel_length,shared_ratio,alpha,epsilon)
estimator.set_params(device=device)
estimator.initialize()  
estimator.module.load_state_dict(torch.load(pre_save_path))
subject_accs = {}

# 逐一验证每个被试
for test_subj in test_subjects:
    # 获取该被试的数据
    X, y, meta = Bench_paradigm.get_data(
        Bench_dataset,
        subjects=[test_subj],
        return_concat=True,
        n_jobs=None,
        verbose=False
    )
    
    # 直接用预训练模型预测
    preds = estimator.predict(X)
    acc = np.mean(preds == y)
    subject_accs[str(test_subj)] = acc

# 先把每个被试的准确率转成百分比并保留一位小数
acc_list = [round(acc * 100, 1) for acc in subject_accs.values()]
# 逐一打印单个被试
for subj, acc in zip(test_subjects, acc_list):
    print(f"subject {subj} 的准确率: {acc:.1f}%")

# 打印所有被试准确率（字典仍是原始小数）
print("所有被试的准确率：", subject_accs)

# 基于百分比后的数值再算平均
mean_acc = np.mean(acc_list)
print(f"所有被试的平均准确率: {mean_acc:.1f}%")

# 额外一行：只显示所有被试准确率（百分数），逗号隔开
print("所有被试准确率（无名称）:", ",".join([f"{acc:.1f}" for acc in acc_list]))

cuda
--------ssssss, /upload/yijun/S22.mat.7z
--------ssssss, /upload/yijun/S24.mat.7z
--------ssssss, /upload/yijun/S27.mat.7z
--------ssssss, /upload/yijun/S28.mat.7z
--------ssssss, /upload/yijun/S31.mat.7z
--------ssssss, /upload/yijun/S34.mat.7z
--------ssssss, /upload/yijun/S35.mat.7z
subject 22 的准确率: 76.7%
subject 24 的准确率: 80.0%
subject 27 的准确率: 81.7%
subject 28 的准确率: 68.8%
subject 31 的准确率: 75.0%
subject 34 的准确率: 67.5%
subject 35 的准确率: 82.1%
所有被试的准确率： {'22': 0.7666666666666667, '24': 0.8, '27': 0.8166666666666667, '28': 0.6875, '31': 0.75, '34': 0.675, '35': 0.8208333333333333}
所有被试的平均准确率: 76.0%
所有被试准确率（无名称）: 76.7,80.0,81.7,68.8,75.0,67.5,82.1


In [3]:
#**************************************************
# Benchmark数据集  EEGNET网络
# !python3.8 Bench_freezing.py # Bench 1e-4学习率，S1~S20  F1=40,D=2,F2=80,Epsilon=20,shared_ratio=0.3,alpha=1.0
# 所有折准确率: [0.68125, 0.6325, 0.6075, 0.6575, 0.655, 0.64375]
# 平均准确率： 0.6462500000000001
#**************************************************
# 设置device（如果GPU可用则使用GPU，否则使用CPU）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
# 设置随机种子
set_random_seeds(42)
kfold = 6
print(X_Bench.shape)  # (2160, 9, 1250)
indices = generate_kfold_indices(meta_Bench, kfold=kfold)

# 预训练阶段
F1 = 20
D = 2
F2 = 40
n_channels = X_Bench.shape[1] # 输入信号的通道数
n_samples = X_Bench.shape[2] # 输入信号的采样点数，时长*采样率
n_classes = 40
time_kernel_length = 50
shared_ratio = 0.2
alpha = 0
epsilon = 30

Bench_save_path = "Bench_eegnet_Sparse_dropout_1s_0_0.2.pth"

pre_accs = []
best_test_acc = -1
best_state_dict = None

for k in range(kfold):
    # 初始化模型
    estimator = EEGNet_Sparse(F1,D,F2,n_channels,n_samples,n_classes,time_kernel_length,shared_ratio,alpha,epsilon)
    estimator.set_params(optimizer__lr=1e-3,device=device) # optimizer__lr=3e-3,
    train_ind, validate_ind, test_ind = match_kfold_indices(k, meta_Bench, indices)
    # 合并train_ind和validate_ind作为新的训练集索引
    new_train_ind = np.concatenate([train_ind, validate_ind])
    new_valid_ind = test_ind  # 用test_ind作为验证集
    new_test_ind = test_ind   # 用test_ind作为测试集

    print(f"train_ind数量: {len(new_train_ind)}, validate_ind数量: {len(new_valid_ind)}, test_ind数量: {len(new_test_ind)}")
    valid_ds = torch.utils.data.TensorDataset(
        torch.tensor(X_Bench[new_valid_ind], dtype=torch.float64),
        torch.tensor(y_Bench[new_valid_ind], dtype=torch.long)
    )
    estimator.set_params(train_split=predefined_split(valid_ds))
    estimator.fit(X_Bench[new_train_ind], y_Bench[new_train_ind])
    # 测试集评估
    p_labels = estimator.predict(X_Bench[new_test_ind])
    p_freqs = [freq_map[label] for label in p_labels]
    y_freqs = [freq_map[label] for label in y_Bench[new_test_ind]]
    print("预测频率：", p_freqs)
    print("真实频率：", y_freqs)
    pre_accs.append(np.mean(p_labels == y_Bench[new_test_ind]))
    print(f"第{k+1}折准确率: {pre_accs[-1]}")
    # 保存最优模型
    if pre_accs[-1] > best_test_acc:
        best_test_acc = pre_accs[-1]
        best_state_dict = copy.deepcopy(estimator.module.state_dict())
print(f"所有折准确率: {pre_accs}")
print("平均准确率：", np.mean(pre_accs))
# 保存测试集上最优的模型
if best_state_dict is not None:
    torch.save(best_state_dict, Bench_save_path)

cuda
(4800, 9, 250)
train_ind数量: 4000, validate_ind数量: 800, test_ind数量: 800
  epoch    train_acc    train_loss    valid_acc    valid_loss    cp      lr     dur
-------  -----------  ------------  -----------  ------------  ----  ------  ------
      1       0.0272        3.8337       0.0312        3.6860     +  0.0010  0.9914
      2       0.0343        3.7332       0.0462        3.6691     +  0.0010  0.7666
      3       0.0382        3.6987       0.0625        3.6499     +  0.0010  0.7524
      4       0.0432        3.6672       0.0875        3.6231     +  0.0010  0.7525
      5       0.0612        3.6340       0.1050        3.5930     +  0.0010  0.7664
      6       0.0693        3.6027       0.1212        3.5514     +  0.0010  0.7578
      7       0.0872        3.5491       0.1625        3.4958     +  0.0010  0.7709
      8       0.1217        3.4826       0.1862        3.4187     +  0.0010  0.7709
      9       0.1415        3.4198       0.2013        3.3263     +  0.0010  0.7495


In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
test_subjects = [22,24,27,28,31,34,35] # 测试使用 
pre_save_path = "Bench_eegnet_Sparse_dropout_1s_0_0.2.pth"
# 加载预训练模型
F1 = 20
D = 2
F2 = 40
n_channels = X_Bench.shape[1] # 输入信号的通道数
n_samples = X_Bench.shape[2] # 输入信号的采样点数，时长*采样率
n_classes = 40
time_kernel_length = 50
shared_ratio = 0.2
alpha = 0
epsilon = 30
estimator = EEGNet_Sparse(F1,D,F2,n_channels,n_samples,n_classes,time_kernel_length,shared_ratio,alpha,epsilon)
estimator.set_params(device=device)
estimator.initialize()  
estimator.module.load_state_dict(torch.load(pre_save_path))
subject_accs = {}

# 逐一验证每个被试
for test_subj in test_subjects:
    # 获取该被试的数据
    X, y, meta = Bench_paradigm.get_data(
        Bench_dataset,
        subjects=[test_subj],
        return_concat=True,
        n_jobs=None,
        verbose=False
    )
    
    # 直接用预训练模型预测
    preds = estimator.predict(X)
    acc = np.mean(preds == y)
    subject_accs[str(test_subj)] = acc

# 先把每个被试的准确率转成百分比并保留一位小数
acc_list = [round(acc * 100, 1) for acc in subject_accs.values()]
# 逐一打印单个被试
for subj, acc in zip(test_subjects, acc_list):
    print(f"subject {subj} 的准确率: {acc:.1f}%")

# 打印所有被试准确率（字典仍是原始小数）
print("所有被试的准确率：", subject_accs)

# 基于百分比后的数值再算平均
mean_acc = np.mean(acc_list)
print(f"所有被试的平均准确率: {mean_acc:.1f}%")

# 额外一行：只显示所有被试准确率（百分数），逗号隔开
print("所有被试准确率（无名称）:", ",".join([f"{acc:.1f}" for acc in acc_list]))

cuda
--------ssssss, /upload/yijun/S22.mat.7z
--------ssssss, /upload/yijun/S24.mat.7z
--------ssssss, /upload/yijun/S27.mat.7z
--------ssssss, /upload/yijun/S28.mat.7z
--------ssssss, /upload/yijun/S31.mat.7z
--------ssssss, /upload/yijun/S34.mat.7z
--------ssssss, /upload/yijun/S35.mat.7z
subject 22 的准确率: 68.8%
subject 24 的准确率: 72.9%
subject 27 的准确率: 81.7%
subject 28 的准确率: 65.8%
subject 31 的准确率: 77.1%
subject 34 的准确率: 61.7%
subject 35 的准确率: 77.9%
所有被试的准确率： {'22': 0.6875, '24': 0.7291666666666666, '27': 0.8166666666666667, '28': 0.6583333333333333, '31': 0.7708333333333334, '34': 0.6166666666666667, '35': 0.7791666666666667}
所有被试的平均准确率: 72.3%
所有被试准确率（无名称）: 68.8,72.9,81.7,65.8,77.1,61.7,77.9


In [16]:
#**************************************************
# Benchmark数据集  EEGNET网络
# !python3.8 Bench_freezing.py # Bench 1e-4学习率，S1~S20  F1=40,D=2,F2=80,Epsilon=20,shared_ratio=0.3,alpha=1.0
# 所有折准确率: [0.68125, 0.6325, 0.6075, 0.6575, 0.655, 0.64375]
# 平均准确率： 0.6462500000000001
#**************************************************
# 设置device（如果GPU可用则使用GPU，否则使用CPU）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
# 设置随机种子
set_random_seeds(42)
kfold = 6
print(X_Bench.shape)  # (2160, 9, 1250)
indices = generate_kfold_indices(meta_Bench, kfold=kfold)

# 预训练阶段
F1 = 20
D = 2
F2 = 40
n_channels = X_Bench.shape[1] # 输入信号的通道数
n_samples = X_Bench.shape[2] # 输入信号的采样点数，时长*采样率
n_classes = 40
time_kernel_length = 50
shared_ratio = 0.3
alpha = 0
epsilon = 30

Bench_save_path = "Bench_eegnet_Sparse_dropout_1s_0_0.3.pth"

pre_accs = []
best_test_acc = -1
best_state_dict = None

for k in range(kfold):
    # 初始化模型
    estimator = EEGNet_Sparse(F1,D,F2,n_channels,n_samples,n_classes,time_kernel_length,shared_ratio,alpha,epsilon)
    estimator.set_params(optimizer__lr=1e-3,device=device) # optimizer__lr=3e-3,
    train_ind, validate_ind, test_ind = match_kfold_indices(k, meta_Bench, indices)
    # 合并train_ind和validate_ind作为新的训练集索引
    new_train_ind = np.concatenate([train_ind, validate_ind])
    new_valid_ind = test_ind  # 用test_ind作为验证集
    new_test_ind = test_ind   # 用test_ind作为测试集

    print(f"train_ind数量: {len(new_train_ind)}, validate_ind数量: {len(new_valid_ind)}, test_ind数量: {len(new_test_ind)}")
    valid_ds = torch.utils.data.TensorDataset(
        torch.tensor(X_Bench[new_valid_ind], dtype=torch.float64),
        torch.tensor(y_Bench[new_valid_ind], dtype=torch.long)
    )
    estimator.set_params(train_split=predefined_split(valid_ds))
    estimator.fit(X_Bench[new_train_ind], y_Bench[new_train_ind])
    # 测试集评估
    p_labels = estimator.predict(X_Bench[new_test_ind])
    p_freqs = [freq_map[label] for label in p_labels]
    y_freqs = [freq_map[label] for label in y_Bench[new_test_ind]]
    print("预测频率：", p_freqs)
    print("真实频率：", y_freqs)
    pre_accs.append(np.mean(p_labels == y_Bench[new_test_ind]))
    print(f"第{k+1}折准确率: {pre_accs[-1]}")
    # 保存最优模型
    if pre_accs[-1] > best_test_acc:
        best_test_acc = pre_accs[-1]
        best_state_dict = copy.deepcopy(estimator.module.state_dict())
print(f"所有折准确率: {pre_accs}")
print("平均准确率：", np.mean(pre_accs))
# 保存测试集上最优的模型
if best_state_dict is not None:
    torch.save(best_state_dict, Bench_save_path)

cuda
(4800, 9, 250)
train_ind数量: 4000, validate_ind数量: 800, test_ind数量: 800
  epoch    train_acc    train_loss    valid_acc    valid_loss    cp      lr     dur
-------  -----------  ------------  -----------  ------------  ----  ------  ------
      1       0.0290        3.8228       0.0275        3.6856     +  0.0010  1.0909
      2       0.0318        3.7295       0.0350        3.6715     +  0.0010  0.7768
      3       0.0370        3.6982       0.0625        3.6556     +  0.0010  0.8306
      4       0.0410        3.6756       0.0775        3.6336     +  0.0010  0.7546
      5       0.0520        3.6456       0.0900        3.6099     +  0.0010  0.8289
      6       0.0673        3.6150       0.1175        3.5774     +  0.0010  0.7637
      7       0.0833        3.5719       0.1388        3.5352     +  0.0010  0.7954
      8       0.1060        3.5259       0.1562        3.4780     +  0.0010  0.7645
      9       0.1258        3.4748       0.1825        3.4057     +  0.0010  0.7780


In [17]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
test_subjects = [22,24,27,28,31,34,35] # 测试使用 
pre_save_path = "Bench_eegnet_Sparse_dropout_1s_0_0.3.pth"
# 加载预训练模型
F1 = 20
D = 2
F2 = 40
n_channels = X_Bench.shape[1] # 输入信号的通道数
n_samples = X_Bench.shape[2] # 输入信号的采样点数，时长*采样率
n_classes = 40
time_kernel_length = 50
shared_ratio = 0.3
alpha = 0
epsilon = 30
estimator = EEGNet_Sparse(F1,D,F2,n_channels,n_samples,n_classes,time_kernel_length,shared_ratio,alpha,epsilon)
estimator.set_params(device=device)
estimator.initialize()  
estimator.module.load_state_dict(torch.load(pre_save_path))
subject_accs = {}

# 逐一验证每个被试
for test_subj in test_subjects:
    # 获取该被试的数据
    X, y, meta = Bench_paradigm.get_data(
        Bench_dataset,
        subjects=[test_subj],
        return_concat=True,
        n_jobs=None,
        verbose=False
    )
    
    # 直接用预训练模型预测
    preds = estimator.predict(X)
    acc = np.mean(preds == y)
    subject_accs[str(test_subj)] = acc

# 先把每个被试的准确率转成百分比并保留一位小数
acc_list = [round(acc * 100, 1) for acc in subject_accs.values()]
# 逐一打印单个被试
for subj, acc in zip(test_subjects, acc_list):
    print(f"subject {subj} 的准确率: {acc:.1f}%")

# 打印所有被试准确率（字典仍是原始小数）
print("所有被试的准确率：", subject_accs)

# 基于百分比后的数值再算平均
mean_acc = np.mean(acc_list)
print(f"所有被试的平均准确率: {mean_acc:.1f}%")

# 额外一行：只显示所有被试准确率（百分数），逗号隔开
print("所有被试准确率（无名称）:", ",".join([f"{acc:.1f}" for acc in acc_list]))

cuda
--------ssssss, /upload/yijun/S22.mat.7z
--------ssssss, /upload/yijun/S24.mat.7z
--------ssssss, /upload/yijun/S27.mat.7z
--------ssssss, /upload/yijun/S28.mat.7z
--------ssssss, /upload/yijun/S31.mat.7z
--------ssssss, /upload/yijun/S34.mat.7z
--------ssssss, /upload/yijun/S35.mat.7z
subject 22 的准确率: 67.9%
subject 24 的准确率: 74.2%
subject 27 的准确率: 80.0%
subject 28 的准确率: 63.3%
subject 31 的准确率: 71.7%
subject 34 的准确率: 65.8%
subject 35 的准确率: 75.8%
所有被试的准确率： {'22': 0.6791666666666667, '24': 0.7416666666666667, '27': 0.8, '28': 0.6333333333333333, '31': 0.7166666666666667, '34': 0.6583333333333333, '35': 0.7583333333333333}
所有被试的平均准确率: 71.2%
所有被试准确率（无名称）: 67.9,74.2,80.0,63.3,71.7,65.8,75.8


In [5]:
#**************************************************
# Benchmark数据集  EEGNET网络
# !python3.8 Bench_freezing.py # Bench 1e-4学习率，S1~S20  F1=40,D=2,F2=80,Epsilon=20,shared_ratio=0.3,alpha=1.0
# 所有折准确率: [0.68125, 0.6325, 0.6075, 0.6575, 0.655, 0.64375]
# 平均准确率： 0.6462500000000001
#**************************************************
# 设置device（如果GPU可用则使用GPU，否则使用CPU）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
# 设置随机种子
set_random_seeds(42)
kfold = 6
print(X_Bench.shape)  # (2160, 9, 1250)
indices = generate_kfold_indices(meta_Bench, kfold=kfold)

# 预训练阶段
F1 = 20
D = 2
F2 = 40
n_channels = X_Bench.shape[1] # 输入信号的通道数
n_samples = X_Bench.shape[2] # 输入信号的采样点数，时长*采样率
n_classes = 40
time_kernel_length = 50
shared_ratio = 0.4
alpha = 0
epsilon = 30

Bench_save_path = "Bench_eegnet_Sparse_dropout_1s_0_0.4.pth"

pre_accs = []
best_test_acc = -1
best_state_dict = None

for k in range(kfold):
    # 初始化模型
    estimator = EEGNet_Sparse(F1,D,F2,n_channels,n_samples,n_classes,time_kernel_length,shared_ratio,alpha,epsilon)
    estimator.set_params(optimizer__lr=1e-3,device=device) # optimizer__lr=3e-3,
    train_ind, validate_ind, test_ind = match_kfold_indices(k, meta_Bench, indices)
    # 合并train_ind和validate_ind作为新的训练集索引
    new_train_ind = np.concatenate([train_ind, validate_ind])
    new_valid_ind = test_ind  # 用test_ind作为验证集
    new_test_ind = test_ind   # 用test_ind作为测试集

    print(f"train_ind数量: {len(new_train_ind)}, validate_ind数量: {len(new_valid_ind)}, test_ind数量: {len(new_test_ind)}")
    valid_ds = torch.utils.data.TensorDataset(
        torch.tensor(X_Bench[new_valid_ind], dtype=torch.float64),
        torch.tensor(y_Bench[new_valid_ind], dtype=torch.long)
    )
    estimator.set_params(train_split=predefined_split(valid_ds))
    estimator.fit(X_Bench[new_train_ind], y_Bench[new_train_ind])
    # 测试集评估
    p_labels = estimator.predict(X_Bench[new_test_ind])
    p_freqs = [freq_map[label] for label in p_labels]
    y_freqs = [freq_map[label] for label in y_Bench[new_test_ind]]
    print("预测频率：", p_freqs)
    print("真实频率：", y_freqs)
    pre_accs.append(np.mean(p_labels == y_Bench[new_test_ind]))
    print(f"第{k+1}折准确率: {pre_accs[-1]}")
    # 保存最优模型
    if pre_accs[-1] > best_test_acc:
        best_test_acc = pre_accs[-1]
        best_state_dict = copy.deepcopy(estimator.module.state_dict())
print(f"所有折准确率: {pre_accs}")
print("平均准确率：", np.mean(pre_accs))
# 保存测试集上最优的模型
if best_state_dict is not None:
    torch.save(best_state_dict, Bench_save_path)

cuda
(4800, 9, 250)
train_ind数量: 4000, validate_ind数量: 800, test_ind数量: 800
  epoch    train_acc    train_loss    valid_acc    valid_loss    cp      lr     dur
-------  -----------  ------------  -----------  ------------  ----  ------  ------
      1       0.0285        3.8024       0.0300        3.6852     +  0.0010  1.0269
      2       0.0328        3.7214       0.0338        3.6715     +  0.0010  0.7465
      3       0.0365        3.6947       0.0600        3.6565     +  0.0010  0.7598
      4       0.0408        3.6774       0.0800        3.6360     +  0.0010  0.7424
      5       0.0545        3.6518       0.0887        3.6119     +  0.0010  0.7308
      6       0.0635        3.6198       0.1163        3.5779     +  0.0010  0.7374
      7       0.0757        3.5795       0.1437        3.5335     +  0.0010  0.7457
      8       0.1030        3.5312       0.1625        3.4804     +  0.0010  0.7567
      9       0.1270        3.4858       0.1775        3.4157     +  0.0010  0.7440


In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
test_subjects = [22,24,27,28,31,34,35] # 测试使用 
pre_save_path = "Bench_eegnet_Sparse_dropout_1s_0_0.4.pth"
# 加载预训练模型
F1 = 20
D = 2
F2 = 40
n_channels = X_Bench.shape[1] # 输入信号的通道数
n_samples = X_Bench.shape[2] # 输入信号的采样点数，时长*采样率
n_classes = 40
time_kernel_length = 50
shared_ratio = 0.4
alpha = 0
epsilon = 30
estimator = EEGNet_Sparse(F1,D,F2,n_channels,n_samples,n_classes,time_kernel_length,shared_ratio,alpha,epsilon)
estimator.set_params(device=device)
estimator.initialize()  
estimator.module.load_state_dict(torch.load(pre_save_path))
subject_accs = {}

# 逐一验证每个被试
for test_subj in test_subjects:
    # 获取该被试的数据
    X, y, meta = Bench_paradigm.get_data(
        Bench_dataset,
        subjects=[test_subj],
        return_concat=True,
        n_jobs=None,
        verbose=False
    )
    
    # 直接用预训练模型预测
    preds = estimator.predict(X)
    acc = np.mean(preds == y)
    subject_accs[str(test_subj)] = acc

# 先把每个被试的准确率转成百分比并保留一位小数
acc_list = [round(acc * 100, 1) for acc in subject_accs.values()]
# 逐一打印单个被试
for subj, acc in zip(test_subjects, acc_list):
    print(f"subject {subj} 的准确率: {acc:.1f}%")

# 打印所有被试准确率（字典仍是原始小数）
print("所有被试的准确率：", subject_accs)

# 基于百分比后的数值再算平均
mean_acc = np.mean(acc_list)
print(f"所有被试的平均准确率: {mean_acc:.1f}%")

# 额外一行：只显示所有被试准确率（百分数），逗号隔开
print("所有被试准确率（无名称）:", ",".join([f"{acc:.1f}" for acc in acc_list]))

cuda
--------ssssss, /upload/yijun/S22.mat.7z
--------ssssss, /upload/yijun/S24.mat.7z
--------ssssss, /upload/yijun/S27.mat.7z
--------ssssss, /upload/yijun/S28.mat.7z
--------ssssss, /upload/yijun/S31.mat.7z
--------ssssss, /upload/yijun/S34.mat.7z
--------ssssss, /upload/yijun/S35.mat.7z
subject 22 的准确率: 65.4%
subject 24 的准确率: 67.1%
subject 27 的准确率: 71.2%
subject 28 的准确率: 63.3%
subject 31 的准确率: 75.4%
subject 34 的准确率: 64.2%
subject 35 的准确率: 76.2%
所有被试的准确率： {'22': 0.6541666666666667, '24': 0.6708333333333333, '27': 0.7125, '28': 0.6333333333333333, '31': 0.7541666666666667, '34': 0.6416666666666667, '35': 0.7625}
所有被试的平均准确率: 69.0%
所有被试准确率（无名称）: 65.4,67.1,71.2,63.3,75.4,64.2,76.2


In [18]:
#**************************************************
# Benchmark数据集  EEGNET网络
# !python3.8 Bench_freezing.py # Bench 1e-4学习率，S1~S20  F1=40,D=2,F2=80,Epsilon=20,shared_ratio=0.3,alpha=1.0
# 所有折准确率: [0.68125, 0.6325, 0.6075, 0.6575, 0.655, 0.64375]
# 平均准确率： 0.6462500000000001
#**************************************************
# 设置device（如果GPU可用则使用GPU，否则使用CPU）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
# 设置随机种子
set_random_seeds(42)
kfold = 6
print(X_Bench.shape)  # (2160, 9, 1250)
indices = generate_kfold_indices(meta_Bench, kfold=kfold)

# 预训练阶段
F1 = 20
D = 2
F2 = 40
n_channels = X_Bench.shape[1] # 输入信号的通道数
n_samples = X_Bench.shape[2] # 输入信号的采样点数，时长*采样率
n_classes = 40
time_kernel_length = 50
shared_ratio = 0.5
alpha = 0
epsilon = 30

Bench_save_path = "Bench_eegnet_Sparse_dropout_1s_0_0.5.pth"

pre_accs = []
best_test_acc = -1
best_state_dict = None

for k in range(kfold):
    # 初始化模型
    estimator = EEGNet_Sparse(F1,D,F2,n_channels,n_samples,n_classes,time_kernel_length,shared_ratio,alpha,epsilon)
    estimator.set_params(optimizer__lr=1e-3,device=device) # optimizer__lr=3e-3,
    train_ind, validate_ind, test_ind = match_kfold_indices(k, meta_Bench, indices)
    # 合并train_ind和validate_ind作为新的训练集索引
    new_train_ind = np.concatenate([train_ind, validate_ind])
    new_valid_ind = test_ind  # 用test_ind作为验证集
    new_test_ind = test_ind   # 用test_ind作为测试集

    print(f"train_ind数量: {len(new_train_ind)}, validate_ind数量: {len(new_valid_ind)}, test_ind数量: {len(new_test_ind)}")
    valid_ds = torch.utils.data.TensorDataset(
        torch.tensor(X_Bench[new_valid_ind], dtype=torch.float64),
        torch.tensor(y_Bench[new_valid_ind], dtype=torch.long)
    )
    estimator.set_params(train_split=predefined_split(valid_ds))
    estimator.fit(X_Bench[new_train_ind], y_Bench[new_train_ind])
    # 测试集评估
    p_labels = estimator.predict(X_Bench[new_test_ind])
    p_freqs = [freq_map[label] for label in p_labels]
    y_freqs = [freq_map[label] for label in y_Bench[new_test_ind]]
    print("预测频率：", p_freqs)
    print("真实频率：", y_freqs)
    pre_accs.append(np.mean(p_labels == y_Bench[new_test_ind]))
    print(f"第{k+1}折准确率: {pre_accs[-1]}")
    # 保存最优模型
    if pre_accs[-1] > best_test_acc:
        best_test_acc = pre_accs[-1]
        best_state_dict = copy.deepcopy(estimator.module.state_dict())
print(f"所有折准确率: {pre_accs}")
print("平均准确率：", np.mean(pre_accs))
# 保存测试集上最优的模型
if best_state_dict is not None:
    torch.save(best_state_dict, Bench_save_path)

cuda
(4800, 9, 250)
train_ind数量: 4000, validate_ind数量: 800, test_ind数量: 800
  epoch    train_acc    train_loss    valid_acc    valid_loss    cp      lr     dur
-------  -----------  ------------  -----------  ------------  ----  ------  ------
      1       0.0222        3.7875       0.0175        3.6860     +  0.0010  0.7593
      2       0.0315        3.7169       0.0375        3.6742     +  0.0010  0.7566
      3       0.0360        3.6946       0.0475        3.6631     +  0.0010  0.7793
      4       0.0355        3.6873       0.0838        3.6487     +  0.0010  0.7475
      5       0.0465        3.6682       0.0988        3.6315     +  0.0010  0.7436
      6       0.0555        3.6452       0.1100        3.6094     +  0.0010  0.7418
      7       0.0620        3.6208       0.1113        3.5845     +  0.0010  0.7544
      8       0.0833        3.5879       0.1250        3.5565     +  0.0010  0.7624
      9       0.0963        3.5629       0.1350        3.5196     +  0.0010  0.7587


In [19]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
test_subjects = [22,24,27,28,31,34,35] # 测试使用 
pre_save_path = "Bench_eegnet_Sparse_dropout_1s_0_0.5.pth"
# 加载预训练模型
F1 = 20
D = 2
F2 = 40
n_channels = X_Bench.shape[1] # 输入信号的通道数
n_samples = X_Bench.shape[2] # 输入信号的采样点数，时长*采样率
n_classes = 40
time_kernel_length = 50
shared_ratio = 0.5
alpha = 0
epsilon = 30
estimator = EEGNet_Sparse(F1,D,F2,n_channels,n_samples,n_classes,time_kernel_length,shared_ratio,alpha,epsilon)
estimator.set_params(device=device)
estimator.initialize()  
estimator.module.load_state_dict(torch.load(pre_save_path))
subject_accs = {}

# 逐一验证每个被试
for test_subj in test_subjects:
    # 获取该被试的数据
    X, y, meta = Bench_paradigm.get_data(
        Bench_dataset,
        subjects=[test_subj],
        return_concat=True,
        n_jobs=None,
        verbose=False
    )
    
    # 直接用预训练模型预测
    preds = estimator.predict(X)
    acc = np.mean(preds == y)
    subject_accs[str(test_subj)] = acc

# 先把每个被试的准确率转成百分比并保留一位小数
acc_list = [round(acc * 100, 1) for acc in subject_accs.values()]
# 逐一打印单个被试
for subj, acc in zip(test_subjects, acc_list):
    print(f"subject {subj} 的准确率: {acc:.1f}%")

# 打印所有被试准确率（字典仍是原始小数）
print("所有被试的准确率：", subject_accs)

# 基于百分比后的数值再算平均
mean_acc = np.mean(acc_list)
print(f"所有被试的平均准确率: {mean_acc:.1f}%")

# 额外一行：只显示所有被试准确率（百分数），逗号隔开
print("所有被试准确率（无名称）:", ",".join([f"{acc:.1f}" for acc in acc_list]))

cuda
--------ssssss, /upload/yijun/S22.mat.7z
--------ssssss, /upload/yijun/S24.mat.7z
--------ssssss, /upload/yijun/S27.mat.7z
--------ssssss, /upload/yijun/S28.mat.7z
--------ssssss, /upload/yijun/S31.mat.7z
--------ssssss, /upload/yijun/S34.mat.7z
--------ssssss, /upload/yijun/S35.mat.7z
subject 22 的准确率: 61.7%
subject 24 的准确率: 68.3%
subject 27 的准确率: 70.4%
subject 28 的准确率: 56.2%
subject 31 的准确率: 66.2%
subject 34 的准确率: 59.6%
subject 35 的准确率: 75.0%
所有被试的准确率： {'22': 0.6166666666666667, '24': 0.6833333333333333, '27': 0.7041666666666667, '28': 0.5625, '31': 0.6625, '34': 0.5958333333333333, '35': 0.75}
所有被试的平均准确率: 65.3%
所有被试准确率（无名称）: 61.7,68.3,70.4,56.2,66.2,59.6,75.0


In [7]:
#**************************************************
# Benchmark数据集  EEGNET网络
# !python3.8 Bench_freezing.py # Bench 1e-4学习率，S1~S20  F1=40,D=2,F2=80,Epsilon=20,shared_ratio=0.3,alpha=1.0
# 所有折准确率: [0.68125, 0.6325, 0.6075, 0.6575, 0.655, 0.64375]
# 平均准确率： 0.6462500000000001
#**************************************************
# 设置device（如果GPU可用则使用GPU，否则使用CPU）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
# 设置随机种子
set_random_seeds(42)
kfold = 6
print(X_Bench.shape)  # (2160, 9, 1250)
indices = generate_kfold_indices(meta_Bench, kfold=kfold)

# 预训练阶段
F1 = 20
D = 2
F2 = 40
n_channels = X_Bench.shape[1] # 输入信号的通道数
n_samples = X_Bench.shape[2] # 输入信号的采样点数，时长*采样率
n_classes = 40
time_kernel_length = 50
shared_ratio = 0.6
alpha = 0
epsilon = 30

Bench_save_path = "Bench_eegnet_Sparse_dropout_1s_0_0.6.pth"

pre_accs = []
best_test_acc = -1
best_state_dict = None

for k in range(kfold):
    # 初始化模型
    estimator = EEGNet_Sparse(F1,D,F2,n_channels,n_samples,n_classes,time_kernel_length,shared_ratio,alpha,epsilon)
    estimator.set_params(optimizer__lr=1e-3,device=device) # optimizer__lr=3e-3,
    train_ind, validate_ind, test_ind = match_kfold_indices(k, meta_Bench, indices)
    # 合并train_ind和validate_ind作为新的训练集索引
    new_train_ind = np.concatenate([train_ind, validate_ind])
    new_valid_ind = test_ind  # 用test_ind作为验证集
    new_test_ind = test_ind   # 用test_ind作为测试集

    print(f"train_ind数量: {len(new_train_ind)}, validate_ind数量: {len(new_valid_ind)}, test_ind数量: {len(new_test_ind)}")
    valid_ds = torch.utils.data.TensorDataset(
        torch.tensor(X_Bench[new_valid_ind], dtype=torch.float64),
        torch.tensor(y_Bench[new_valid_ind], dtype=torch.long)
    )
    estimator.set_params(train_split=predefined_split(valid_ds))
    estimator.fit(X_Bench[new_train_ind], y_Bench[new_train_ind])
    # 测试集评估
    p_labels = estimator.predict(X_Bench[new_test_ind])
    p_freqs = [freq_map[label] for label in p_labels]
    y_freqs = [freq_map[label] for label in y_Bench[new_test_ind]]
    print("预测频率：", p_freqs)
    print("真实频率：", y_freqs)
    pre_accs.append(np.mean(p_labels == y_Bench[new_test_ind]))
    print(f"第{k+1}折准确率: {pre_accs[-1]}")
    # 保存最优模型
    if pre_accs[-1] > best_test_acc:
        best_test_acc = pre_accs[-1]
        best_state_dict = copy.deepcopy(estimator.module.state_dict())
print(f"所有折准确率: {pre_accs}")
print("平均准确率：", np.mean(pre_accs))
# 保存测试集上最优的模型
if best_state_dict is not None:
    torch.save(best_state_dict, Bench_save_path)

cuda
(4800, 9, 250)
train_ind数量: 4000, validate_ind数量: 800, test_ind数量: 800
  epoch    train_acc    train_loss    valid_acc    valid_loss    cp      lr     dur
-------  -----------  ------------  -----------  ------------  ----  ------  ------
      1       0.0225        3.7697       0.0187        3.6861     +  0.0010  0.7753
      2       0.0318        3.7170       0.0375        3.6767     +  0.0010  0.7624
      3       0.0305        3.6947       0.0537        3.6678     +  0.0010  0.7669
      4       0.0390        3.6860       0.0700        3.6548     +  0.0010  0.7632
      5       0.0420        3.6702       0.0862        3.6412     +  0.0010  0.7541
      6       0.0480        3.6533       0.0988        3.6225     +  0.0010  0.7580
      7       0.0570        3.6328       0.0975        3.6027     +  0.0010  0.7625
      8       0.0727        3.6060       0.1125        3.5784     +  0.0010  0.7709
      9       0.0820        3.5873       0.1338        3.5495     +  0.0010  0.7666


In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
test_subjects = [22,24,27,28,31,34,35] # 测试使用 
pre_save_path = "Bench_eegnet_Sparse_dropout_1s_0_0.6.pth"
# 加载预训练模型
F1 = 20
D = 2
F2 = 40
n_channels = X_Bench.shape[1] # 输入信号的通道数
n_samples = X_Bench.shape[2] # 输入信号的采样点数，时长*采样率
n_classes = 40
time_kernel_length = 50
shared_ratio = 0.6
alpha = 0
epsilon = 30
estimator = EEGNet_Sparse(F1,D,F2,n_channels,n_samples,n_classes,time_kernel_length,shared_ratio,alpha,epsilon)
estimator.set_params(device=device)
estimator.initialize()  
estimator.module.load_state_dict(torch.load(pre_save_path))
subject_accs = {}

# 逐一验证每个被试
for test_subj in test_subjects:
    # 获取该被试的数据
    X, y, meta = Bench_paradigm.get_data(
        Bench_dataset,
        subjects=[test_subj],
        return_concat=True,
        n_jobs=None,
        verbose=False
    )
    
    # 直接用预训练模型预测
    preds = estimator.predict(X)
    acc = np.mean(preds == y)
    subject_accs[str(test_subj)] = acc

# 先把每个被试的准确率转成百分比并保留一位小数
acc_list = [round(acc * 100, 1) for acc in subject_accs.values()]
# 逐一打印单个被试
for subj, acc in zip(test_subjects, acc_list):
    print(f"subject {subj} 的准确率: {acc:.1f}%")

# 打印所有被试准确率（字典仍是原始小数）
print("所有被试的准确率：", subject_accs)

# 基于百分比后的数值再算平均
mean_acc = np.mean(acc_list)
print(f"所有被试的平均准确率: {mean_acc:.1f}%")

# 额外一行：只显示所有被试准确率（百分数），逗号隔开
print("所有被试准确率（无名称）:", ",".join([f"{acc:.1f}" for acc in acc_list]))

cuda
--------ssssss, /upload/yijun/S22.mat.7z
--------ssssss, /upload/yijun/S24.mat.7z
--------ssssss, /upload/yijun/S27.mat.7z
--------ssssss, /upload/yijun/S28.mat.7z
--------ssssss, /upload/yijun/S31.mat.7z
--------ssssss, /upload/yijun/S34.mat.7z
--------ssssss, /upload/yijun/S35.mat.7z
subject 22 的准确率: 54.6%
subject 24 的准确率: 60.0%
subject 27 的准确率: 62.9%
subject 28 的准确率: 53.3%
subject 31 的准确率: 55.8%
subject 34 的准确率: 50.8%
subject 35 的准确率: 60.0%
所有被试的准确率： {'22': 0.5458333333333333, '24': 0.6, '27': 0.6291666666666667, '28': 0.5333333333333333, '31': 0.5583333333333333, '34': 0.5083333333333333, '35': 0.6}
所有被试的平均准确率: 56.8%
所有被试准确率（无名称）: 54.6,60.0,62.9,53.3,55.8,50.8,60.0


In [20]:
#**************************************************
# Benchmark数据集  EEGNET网络
# !python3.8 Bench_freezing.py # Bench 1e-4学习率，S1~S20  F1=40,D=2,F2=80,Epsilon=20,shared_ratio=0.3,alpha=1.0
# 所有折准确率: [0.68125, 0.6325, 0.6075, 0.6575, 0.655, 0.64375]
# 平均准确率： 0.6462500000000001
#**************************************************
# 设置device（如果GPU可用则使用GPU，否则使用CPU）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
# 设置随机种子
set_random_seeds(42)
kfold = 6
print(X_Bench.shape)  # (2160, 9, 1250)
indices = generate_kfold_indices(meta_Bench, kfold=kfold)

# 预训练阶段
F1 = 20
D = 2
F2 = 40
n_channels = X_Bench.shape[1] # 输入信号的通道数
n_samples = X_Bench.shape[2] # 输入信号的采样点数，时长*采样率
n_classes = 40
time_kernel_length = 50
shared_ratio = 0.7
alpha = 0
epsilon = 30

Bench_save_path = "Bench_eegnet_Sparse_dropout_1s_0_0.7.pth"

pre_accs = []
best_test_acc = -1
best_state_dict = None

for k in range(kfold):
    # 初始化模型
    estimator = EEGNet_Sparse(F1,D,F2,n_channels,n_samples,n_classes,time_kernel_length,shared_ratio,alpha,epsilon)
    estimator.set_params(optimizer__lr=1e-3,device=device) # optimizer__lr=3e-3,
    train_ind, validate_ind, test_ind = match_kfold_indices(k, meta_Bench, indices)
    # 合并train_ind和validate_ind作为新的训练集索引
    new_train_ind = np.concatenate([train_ind, validate_ind])
    new_valid_ind = test_ind  # 用test_ind作为验证集
    new_test_ind = test_ind   # 用test_ind作为测试集

    print(f"train_ind数量: {len(new_train_ind)}, validate_ind数量: {len(new_valid_ind)}, test_ind数量: {len(new_test_ind)}")
    valid_ds = torch.utils.data.TensorDataset(
        torch.tensor(X_Bench[new_valid_ind], dtype=torch.float64),
        torch.tensor(y_Bench[new_valid_ind], dtype=torch.long)
    )
    estimator.set_params(train_split=predefined_split(valid_ds))
    estimator.fit(X_Bench[new_train_ind], y_Bench[new_train_ind])
    # 测试集评估
    p_labels = estimator.predict(X_Bench[new_test_ind])
    p_freqs = [freq_map[label] for label in p_labels]
    y_freqs = [freq_map[label] for label in y_Bench[new_test_ind]]
    print("预测频率：", p_freqs)
    print("真实频率：", y_freqs)
    pre_accs.append(np.mean(p_labels == y_Bench[new_test_ind]))
    print(f"第{k+1}折准确率: {pre_accs[-1]}")
    # 保存最优模型
    if pre_accs[-1] > best_test_acc:
        best_test_acc = pre_accs[-1]
        best_state_dict = copy.deepcopy(estimator.module.state_dict())
print(f"所有折准确率: {pre_accs}")
print("平均准确率：", np.mean(pre_accs))
# 保存测试集上最优的模型
if best_state_dict is not None:
    torch.save(best_state_dict, Bench_save_path)

cuda
(4800, 9, 250)
train_ind数量: 4000, validate_ind数量: 800, test_ind数量: 800
  epoch    train_acc    train_loss    valid_acc    valid_loss    cp      lr     dur
-------  -----------  ------------  -----------  ------------  ----  ------  ------
      1       0.0243        3.7536       0.0200        3.6860     +  0.0010  1.0166
      2       0.0300        3.7114       0.0262        3.6773     +  0.0010  0.7334
      3       0.0302        3.6935       0.0437        3.6698     +  0.0010  0.7565
      4       0.0387        3.6814       0.0625        3.6585     +  0.0010  0.7459
      5       0.0403        3.6715       0.0712        3.6483     +  0.0010  0.7555
      6       0.0437        3.6603       0.0775        3.6337     +  0.0010  0.7545
      7       0.0553        3.6439       0.0900        3.6166     +  0.0010  0.7394
      8       0.0612        3.6245       0.0963        3.5976     +  0.0010  0.7424
      9       0.0678        3.6054       0.1037        3.5746     +  0.0010  0.7477


In [21]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
test_subjects = [22,24,27,28,31,34,35] # 测试使用 
pre_save_path = "Bench_eegnet_Sparse_dropout_1s_0_0.7.pth"
# 加载预训练模型
F1 = 20
D = 2
F2 = 40
n_channels = X_Bench.shape[1] # 输入信号的通道数
n_samples = X_Bench.shape[2] # 输入信号的采样点数，时长*采样率
n_classes = 40
time_kernel_length = 50
shared_ratio = 0.7
alpha = 0
epsilon = 30
estimator = EEGNet_Sparse(F1,D,F2,n_channels,n_samples,n_classes,time_kernel_length,shared_ratio,alpha,epsilon)
estimator.set_params(device=device)
estimator.initialize()  
estimator.module.load_state_dict(torch.load(pre_save_path))
subject_accs = {}

# 逐一验证每个被试
for test_subj in test_subjects:
    # 获取该被试的数据
    X, y, meta = Bench_paradigm.get_data(
        Bench_dataset,
        subjects=[test_subj],
        return_concat=True,
        n_jobs=None,
        verbose=False
    )
    
    # 直接用预训练模型预测
    preds = estimator.predict(X)
    acc = np.mean(preds == y)
    subject_accs[str(test_subj)] = acc

# 先把每个被试的准确率转成百分比并保留一位小数
acc_list = [round(acc * 100, 1) for acc in subject_accs.values()]
# 逐一打印单个被试
for subj, acc in zip(test_subjects, acc_list):
    print(f"subject {subj} 的准确率: {acc:.1f}%")

# 打印所有被试准确率（字典仍是原始小数）
print("所有被试的准确率：", subject_accs)

# 基于百分比后的数值再算平均
mean_acc = np.mean(acc_list)
print(f"所有被试的平均准确率: {mean_acc:.1f}%")

# 额外一行：只显示所有被试准确率（百分数），逗号隔开
print("所有被试准确率（无名称）:", ",".join([f"{acc:.1f}" for acc in acc_list]))

cuda
--------ssssss, /upload/yijun/S22.mat.7z
--------ssssss, /upload/yijun/S24.mat.7z
--------ssssss, /upload/yijun/S27.mat.7z
--------ssssss, /upload/yijun/S28.mat.7z
--------ssssss, /upload/yijun/S31.mat.7z
--------ssssss, /upload/yijun/S34.mat.7z
--------ssssss, /upload/yijun/S35.mat.7z
subject 22 的准确率: 58.8%
subject 24 的准确率: 57.9%
subject 27 的准确率: 63.7%
subject 28 的准确率: 53.3%
subject 31 的准确率: 57.1%
subject 34 的准确率: 55.8%
subject 35 的准确率: 62.5%
所有被试的准确率： {'22': 0.5875, '24': 0.5791666666666667, '27': 0.6375, '28': 0.5333333333333333, '31': 0.5708333333333333, '34': 0.5583333333333333, '35': 0.625}
所有被试的平均准确率: 58.4%
所有被试准确率（无名称）: 58.8,57.9,63.7,53.3,57.1,55.8,62.5


In [9]:
#**************************************************
# Benchmark数据集  EEGNET网络
# !python3.8 Bench_freezing.py # Bench 1e-4学习率，S1~S20  F1=40,D=2,F2=80,Epsilon=20,shared_ratio=0.3,alpha=1.0
# 所有折准确率: [0.68125, 0.6325, 0.6075, 0.6575, 0.655, 0.64375]
# 平均准确率： 0.6462500000000001
#**************************************************
# 设置device（如果GPU可用则使用GPU，否则使用CPU）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
# 设置随机种子
set_random_seeds(42)
kfold = 6
print(X_Bench.shape)  # (2160, 9, 1250)
indices = generate_kfold_indices(meta_Bench, kfold=kfold)

# 预训练阶段
F1 = 20
D = 2
F2 = 40
n_channels = X_Bench.shape[1] # 输入信号的通道数
n_samples = X_Bench.shape[2] # 输入信号的采样点数，时长*采样率
n_classes = 40
time_kernel_length = 50
shared_ratio = 0.8
alpha = 0
epsilon = 30

Bench_save_path = "Bench_eegnet_Sparse_dropout_1s_0_0.8.pth"

pre_accs = []
best_test_acc = -1
best_state_dict = None

for k in range(kfold):
    # 初始化模型
    estimator = EEGNet_Sparse(F1,D,F2,n_channels,n_samples,n_classes,time_kernel_length,shared_ratio,alpha,epsilon)
    estimator.set_params(optimizer__lr=1e-3,device=device) # optimizer__lr=3e-3,
    train_ind, validate_ind, test_ind = match_kfold_indices(k, meta_Bench, indices)
    # 合并train_ind和validate_ind作为新的训练集索引
    new_train_ind = np.concatenate([train_ind, validate_ind])
    new_valid_ind = test_ind  # 用test_ind作为验证集
    new_test_ind = test_ind   # 用test_ind作为测试集

    print(f"train_ind数量: {len(new_train_ind)}, validate_ind数量: {len(new_valid_ind)}, test_ind数量: {len(new_test_ind)}")
    valid_ds = torch.utils.data.TensorDataset(
        torch.tensor(X_Bench[new_valid_ind], dtype=torch.float64),
        torch.tensor(y_Bench[new_valid_ind], dtype=torch.long)
    )
    estimator.set_params(train_split=predefined_split(valid_ds))
    estimator.fit(X_Bench[new_train_ind], y_Bench[new_train_ind])
    # 测试集评估
    p_labels = estimator.predict(X_Bench[new_test_ind])
    p_freqs = [freq_map[label] for label in p_labels]
    y_freqs = [freq_map[label] for label in y_Bench[new_test_ind]]
    print("预测频率：", p_freqs)
    print("真实频率：", y_freqs)
    pre_accs.append(np.mean(p_labels == y_Bench[new_test_ind]))
    print(f"第{k+1}折准确率: {pre_accs[-1]}")
    # 保存最优模型
    if pre_accs[-1] > best_test_acc:
        best_test_acc = pre_accs[-1]
        best_state_dict = copy.deepcopy(estimator.module.state_dict())
print(f"所有折准确率: {pre_accs}")
print("平均准确率：", np.mean(pre_accs))
# 保存测试集上最优的模型
if best_state_dict is not None:
    torch.save(best_state_dict, Bench_save_path)

cuda
(4800, 9, 250)
train_ind数量: 4000, validate_ind数量: 800, test_ind数量: 800
  epoch    train_acc    train_loss    valid_acc    valid_loss    cp      lr     dur
-------  -----------  ------------  -----------  ------------  ----  ------  ------
      1       0.0255        3.7382       0.0275        3.6876     +  0.0010  0.7800
      2       0.0275        3.7060       0.0325        3.6835     +  0.0010  0.7577
      3       0.0257        3.6929       0.0325        3.6791     +  0.0010  0.7297
      4       0.0357        3.6841       0.0325        3.6729     +  0.0010  0.7521
      5       0.0320        3.6829       0.0537        3.6670     +  0.0010  0.7485
      6       0.0372        3.6733       0.0688        3.6595     +  0.0010  0.7547
      7       0.0435        3.6654       0.0750        3.6517     +  0.0010  0.7673
      8       0.0500        3.6525       0.0813        3.6428     +  0.0010  0.7374
      9       0.0570        3.6437       0.0938        3.6311     +  0.0010  0.7414


In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
test_subjects = [22,24,27,28,31,34,35] # 测试使用 
pre_save_path = "Bench_eegnet_Sparse_dropout_1s_0_0.8.pth"
# 加载预训练模型
F1 = 20
D = 2
F2 = 40
n_channels = X_Bench.shape[1] # 输入信号的通道数
n_samples = X_Bench.shape[2] # 输入信号的采样点数，时长*采样率
n_classes = 40
time_kernel_length = 50
shared_ratio = 0.8
alpha = 0
epsilon = 30
estimator = EEGNet_Sparse(F1,D,F2,n_channels,n_samples,n_classes,time_kernel_length,shared_ratio,alpha,epsilon)
estimator.set_params(device=device)
estimator.initialize()  
estimator.module.load_state_dict(torch.load(pre_save_path))
subject_accs = {}

# 逐一验证每个被试
for test_subj in test_subjects:
    # 获取该被试的数据
    X, y, meta = Bench_paradigm.get_data(
        Bench_dataset,
        subjects=[test_subj],
        return_concat=True,
        n_jobs=None,
        verbose=False
    )
    
    # 直接用预训练模型预测
    preds = estimator.predict(X)
    acc = np.mean(preds == y)
    subject_accs[str(test_subj)] = acc

# 先把每个被试的准确率转成百分比并保留一位小数
acc_list = [round(acc * 100, 1) for acc in subject_accs.values()]
# 逐一打印单个被试
for subj, acc in zip(test_subjects, acc_list):
    print(f"subject {subj} 的准确率: {acc:.1f}%")

# 打印所有被试准确率（字典仍是原始小数）
print("所有被试的准确率：", subject_accs)

# 基于百分比后的数值再算平均
mean_acc = np.mean(acc_list)
print(f"所有被试的平均准确率: {mean_acc:.1f}%")

# 额外一行：只显示所有被试准确率（百分数），逗号隔开
print("所有被试准确率（无名称）:", ",".join([f"{acc:.1f}" for acc in acc_list]))

cuda
--------ssssss, /upload/yijun/S22.mat.7z
--------ssssss, /upload/yijun/S24.mat.7z
--------ssssss, /upload/yijun/S27.mat.7z
--------ssssss, /upload/yijun/S28.mat.7z
--------ssssss, /upload/yijun/S31.mat.7z
--------ssssss, /upload/yijun/S34.mat.7z
--------ssssss, /upload/yijun/S35.mat.7z
subject 22 的准确率: 57.1%
subject 24 的准确率: 62.1%
subject 27 的准确率: 67.5%
subject 28 的准确率: 52.9%
subject 31 的准确率: 57.9%
subject 34 的准确率: 54.6%
subject 35 的准确率: 62.1%
所有被试的准确率： {'22': 0.5708333333333333, '24': 0.6208333333333333, '27': 0.675, '28': 0.5291666666666667, '31': 0.5791666666666667, '34': 0.5458333333333333, '35': 0.6208333333333333}
所有被试的平均准确率: 59.2%
所有被试准确率（无名称）: 57.1,62.1,67.5,52.9,57.9,54.6,62.1


In [22]:
#**************************************************
# Benchmark数据集  EEGNET网络
# !python3.8 Bench_freezing.py # Bench 1e-4学习率，S1~S20  F1=40,D=2,F2=80,Epsilon=20,shared_ratio=0.3,alpha=1.0
# 所有折准确率: [0.68125, 0.6325, 0.6075, 0.6575, 0.655, 0.64375]
# 平均准确率： 0.6462500000000001
#**************************************************
# 设置device（如果GPU可用则使用GPU，否则使用CPU）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
# 设置随机种子
set_random_seeds(42)
kfold = 6
print(X_Bench.shape)  # (2160, 9, 1250)
indices = generate_kfold_indices(meta_Bench, kfold=kfold)

# 预训练阶段
F1 = 20
D = 2
F2 = 40
n_channels = X_Bench.shape[1] # 输入信号的通道数
n_samples = X_Bench.shape[2] # 输入信号的采样点数，时长*采样率
n_classes = 40
time_kernel_length = 50
shared_ratio = 0.9
alpha = 0
epsilon = 30

Bench_save_path = "Bench_eegnet_Sparse_dropout_1s_0_0.9.pth"

pre_accs = []
best_test_acc = -1
best_state_dict = None

for k in range(kfold):
    # 初始化模型
    estimator = EEGNet_Sparse(F1,D,F2,n_channels,n_samples,n_classes,time_kernel_length,shared_ratio,alpha,epsilon)
    estimator.set_params(optimizer__lr=1e-3,device=device) # optimizer__lr=3e-3,
    train_ind, validate_ind, test_ind = match_kfold_indices(k, meta_Bench, indices)
    # 合并train_ind和validate_ind作为新的训练集索引
    new_train_ind = np.concatenate([train_ind, validate_ind])
    new_valid_ind = test_ind  # 用test_ind作为验证集
    new_test_ind = test_ind   # 用test_ind作为测试集

    print(f"train_ind数量: {len(new_train_ind)}, validate_ind数量: {len(new_valid_ind)}, test_ind数量: {len(new_test_ind)}")
    valid_ds = torch.utils.data.TensorDataset(
        torch.tensor(X_Bench[new_valid_ind], dtype=torch.float64),
        torch.tensor(y_Bench[new_valid_ind], dtype=torch.long)
    )
    estimator.set_params(train_split=predefined_split(valid_ds))
    estimator.fit(X_Bench[new_train_ind], y_Bench[new_train_ind])
    # 测试集评估
    p_labels = estimator.predict(X_Bench[new_test_ind])
    p_freqs = [freq_map[label] for label in p_labels]
    y_freqs = [freq_map[label] for label in y_Bench[new_test_ind]]
    print("预测频率：", p_freqs)
    print("真实频率：", y_freqs)
    pre_accs.append(np.mean(p_labels == y_Bench[new_test_ind]))
    print(f"第{k+1}折准确率: {pre_accs[-1]}")
    # 保存最优模型
    if pre_accs[-1] > best_test_acc:
        best_test_acc = pre_accs[-1]
        best_state_dict = copy.deepcopy(estimator.module.state_dict())
print(f"所有折准确率: {pre_accs}")
print("平均准确率：", np.mean(pre_accs))
# 保存测试集上最优的模型
if best_state_dict is not None:
    torch.save(best_state_dict, Bench_save_path)

cuda
(4800, 9, 250)
train_ind数量: 4000, validate_ind数量: 800, test_ind数量: 800
  epoch    train_acc    train_loss    valid_acc    valid_loss    cp      lr     dur
-------  -----------  ------------  -----------  ------------  ----  ------  ------
      1       0.0222        3.7146       0.0275        3.6870     +  0.0010  1.0617
      2       0.0275        3.6970       0.0262        3.6842     +  0.0010  0.7624
      3       0.0293        3.6875       0.0400        3.6802     +  0.0010  0.7625
      4       0.0307        3.6831       0.0563        3.6759     +  0.0010  0.7601
      5       0.0335        3.6819       0.0700        3.6717     +  0.0010  0.7333
      6       0.0390        3.6725       0.0762        3.6660     +  0.0010  0.7422
      7       0.0485        3.6663       0.0800        3.6597     +  0.0010  0.7500
      8       0.0512        3.6583       0.0912        3.6520     +  0.0010  0.7501
      9       0.0595        3.6500       0.0988        3.6418     +  0.0010  0.7708


In [23]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
test_subjects = [22,24,27,28,31,34,35] # 测试使用 
pre_save_path = "Bench_eegnet_Sparse_dropout_1s_0_0.9.pth"
# 加载预训练模型
F1 = 20
D = 2
F2 = 40
n_channels = X_Bench.shape[1] # 输入信号的通道数
n_samples = X_Bench.shape[2] # 输入信号的采样点数，时长*采样率
n_classes = 40
time_kernel_length = 50
shared_ratio = 0.9
alpha = 0
epsilon = 30
estimator = EEGNet_Sparse(F1,D,F2,n_channels,n_samples,n_classes,time_kernel_length,shared_ratio,alpha,epsilon)
estimator.set_params(device=device)
estimator.initialize()  
estimator.module.load_state_dict(torch.load(pre_save_path))
subject_accs = {}

# 逐一验证每个被试
for test_subj in test_subjects:
    # 获取该被试的数据
    X, y, meta = Bench_paradigm.get_data(
        Bench_dataset,
        subjects=[test_subj],
        return_concat=True,
        n_jobs=None,
        verbose=False
    )
    
    # 直接用预训练模型预测
    preds = estimator.predict(X)
    acc = np.mean(preds == y)
    subject_accs[str(test_subj)] = acc

# 先把每个被试的准确率转成百分比并保留一位小数
acc_list = [round(acc * 100, 1) for acc in subject_accs.values()]
# 逐一打印单个被试
for subj, acc in zip(test_subjects, acc_list):
    print(f"subject {subj} 的准确率: {acc:.1f}%")

# 打印所有被试准确率（字典仍是原始小数）
print("所有被试的准确率：", subject_accs)

# 基于百分比后的数值再算平均
mean_acc = np.mean(acc_list)
print(f"所有被试的平均准确率: {mean_acc:.1f}%")

# 额外一行：只显示所有被试准确率（百分数），逗号隔开
print("所有被试准确率（无名称）:", ",".join([f"{acc:.1f}" for acc in acc_list]))

cuda
--------ssssss, /upload/yijun/S22.mat.7z
--------ssssss, /upload/yijun/S24.mat.7z
--------ssssss, /upload/yijun/S27.mat.7z
--------ssssss, /upload/yijun/S28.mat.7z
--------ssssss, /upload/yijun/S31.mat.7z
--------ssssss, /upload/yijun/S34.mat.7z
--------ssssss, /upload/yijun/S35.mat.7z
subject 22 的准确率: 44.2%
subject 24 的准确率: 43.3%
subject 27 的准确率: 49.2%
subject 28 的准确率: 36.7%
subject 31 的准确率: 50.4%
subject 34 的准确率: 40.0%
subject 35 的准确率: 53.3%
所有被试的准确率： {'22': 0.44166666666666665, '24': 0.43333333333333335, '27': 0.49166666666666664, '28': 0.36666666666666664, '31': 0.5041666666666667, '34': 0.4, '35': 0.5333333333333333}
所有被试的平均准确率: 45.3%
所有被试准确率（无名称）: 44.2,43.3,49.2,36.7,50.4,40.0,53.3
